# MultiIndex Approach

One common approach is to use a pandas MultiIndex to combine time and metadata into a single dimension.

## Setup

In [9]:
%xmode minimal
import numpy as np
import pandas as pd
import xarray as xr

Exception reporting mode: Minimal


## Creating the MultiIndex Dataset

We create a dataset where the time dimension has a MultiIndex with both time values and word labels.

In [10]:
# Create sample data
T = 1000
C = 2
times = np.linspace(0, 120, T)
data = np.random.rand(C, T)

# Define word boundaries
breaks = np.array([0, 333, 666, 1000])

# Create word labels for each time point
words = np.array(["red"] * T)
words[breaks[0] : breaks[1]] = "red"
words[breaks[1] : breaks[2]] = "green"
words[breaks[2] :] = "blue"

# Create MultiIndex
mdx = pd.MultiIndex.from_arrays([words, times], names=["word", "time"])

# Create xarray Dataset
ds = xr.DataArray(data, [("C", range(C)), ("T", mdx)]).to_dataset(name="data")
ds

<xarray.Dataset> Size: 40kB
Dimensions:  (C: 2, T: 1000)
Coordinates:
  * C        (C) int64 16B 0 1
  * T        (T) object 8kB MultiIndex
  * word     (T) object 8kB 'red' 'red' 'red' 'red' ... 'blu' 'blu' 'blu' 'blu'
  * time     (T) float64 8kB 0.0 0.1201 0.2402 0.3604 ... 119.8 119.9 120.0
Data variables:
    data     (C, T) float64 16kB 0.8339 0.7163 0.4126 ... 0.5775 0.715 0.8221

## Selection by Word

We can select by word label:

In [11]:
ds.sel(word="red")

<xarray.Dataset> Size: 8kB
Dimensions:  (C: 2, time: 333)
Coordinates:
  * C        (C) int64 16B 0 1
  * time     (time) float64 3kB 0.0 0.1201 0.2402 0.3604 ... 39.64 39.76 39.88
    word     <U3 12B 'red'
Data variables:
    data     (C, time) float64 5kB 0.8339 0.7163 0.4126 ... 0.1509 0.816 0.9262

## Limitations

### 1. Time slicing is awkward

Slicing by time requires specifying the word level too, or using `.loc` patterns:

For float valued time slicing raises a ValueError!

In [12]:
# This doesn't work as expected:
ds.sel(time=slice(0, 50))  # TypeError

TypeError: float() argument must be a string or a real number, not 'slice'

In [14]:
# Need to be more explicit
# but this will always return a copy, instead of a less expensive view.
ds.sel(T=ds.T[(ds.time >= 0.15) & (ds.time <= 55.8)])
# ds.sel(T=slice(.15, 55.8))

<xarray.Dataset> Size: 19kB
Dimensions:  (C: 2, T: 463)
Coordinates:
  * C        (C) int64 16B 0 1
  * T        (T) object 4kB MultiIndex
  * word     (T) object 4kB 'red' 'red' 'red' 'red' ... 'gre' 'gre' 'gre' 'gre'
  * time     (T) float64 4kB 0.2402 0.3604 0.4805 0.6006 ... 55.5 55.62 55.74
Data variables:
    data     (C, T) float64 7kB 0.4126 0.2871 0.1941 ... 0.4977 0.3668 0.4986

### 2. can't `isel` the metadata coords

You might want to be able to select the 3rd word. That is not easy with this index.

In [16]:
# value error
ds.isel(word=0)

ValueError: Dimensions {'word'} do not exist. Expected one or more of FrozenMappingWarningOnValuesAccess({'C': 2, 'T': 1000})

### 3. Interval boundaries are lost

The MultiIndex doesn't preserve the actual interval boundaries - we can't easily ask "what time range does word X span?" without computing it from the data.

### 4. Constrained to measurement time points

If metadata events happen at times not in your measurement grid, you lose that precision. For example, if you sample monthly but an event happened mid-month, you can't represent that exactly.
### 5. Memory duplication

Each time point stores its word label, which is redundant information.
